# Performance & Risk Data

In MongoDB, all data of a particular `fund` at a particular `date` is stored in a single document. Values are stored as an Embedded Document and can be nested further.

```javascript
{
    _id: ObjectId('...')
    date: 2023-12-31T00:00:00.000+00:00
    fundId: ObjectId('...')
    values: {
        nav: 100
        region: {
            AMER: 0.3
            APAC: 0.2
            EMEA: 0.1
        }
    }
}
```

Import the libraries and connect to the database

In [5]:
import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
client = MongoClient(os.environ["MONGODB_HOST"])
db = client.ftkdb

## Saving Data

In [44]:
fund1 = db.fund.find_one({'name': 'Fund One'})
fund2 = db.fund.find_one({'name': 'Fund Two'})

# index is Timestamp, row is Series
def save_data(fund, data):
    for date, row in data.iterrows():    
        row.index = row.index.map(lambda x: f'values.{x}')
        result = db.data.update_one({
            'fundId': fund['_id'],
            'date': date
        },{
            '$set': {**row.dropna().to_dict()}
        },
        upsert=True)

In [45]:
dates = pd.date_range('2023-12-31', periods=6, freq='M')
navs = pd.DataFrame(np.random.randint(100, 110, size=6), index=dates, columns=['nav'])
save_data(fund1, navs)

In [46]:
rets = navs.pct_change().to_period()
# Period cannot be encoded
rets = rets.to_timestamp()
rets.index = rets.index + pd.offsets.MonthEnd(0)
rets.columns = ['return']
save_data(fund1, rets)

In [47]:
regions = pd.DataFrame(np.random.rand(6, 3), index=dates, columns=['region.AMER', 'region.EMEA', 'region.APAC'])
save_data(fund1, regions)

In [50]:
multi = pd.concat([navs, rets, regions], axis=1)
save_data(fund2, multi)

## Loading Data

In [57]:
data = list(db.data.find({'fundId': fund1['_id']}).sort({'date': 1}))
df = pd.DataFrame([d['values'] for d in data], index=[d['date'] for d in data])

navs = df.loc[:, 'nav']
regions = df.loc[:, 'region'].apply(pd.Series)